In [ ]:
#| default_exp ChatDB

#  ChatDB
> ChatDB functionality for khazaddum

In [ ]:
#|export
import sqlite3
import json
from datetime import datetime
from typing import List, Dict, Optional

In [ ]:
#|export
class ChatDatabase:
    """Simple database for storing chat sessions with JSON history"""
    
    def __init__(self, db_path: str):
        """
        Initialize the database connection and create tables if needed.
        
        Args:
            db_path: Path to the SQLite database file
        """
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path,  check_same_thread=False)
        self.conn.row_factory = sqlite3.Row
        self._setup_tables()
    
    def _setup_tables(self):
        """Create the chat_sessions table if it doesn't exist"""
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS chat_sessions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_name TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                model_name TEXT,
                history_json TEXT NOT NULL
            )
        ''')
        self.conn.commit()
    
    def save_session(self, session_name: str, history: List[Dict], 
                     model_name: str = None) -> int:
        """
        Save a new chat session.
        
        Args:
            session_name: Name for this session
            history: List of message dictionaries (chat.hist)
            model_name: Name of the model used
            
        Returns:
            The ID of the newly created session
        """
        cursor = self.conn.cursor()
        history_json = json.dumps(history)
        
        cursor.execute('''
            INSERT INTO chat_sessions 
            (session_name, history_json, model_name) 
            VALUES (?, ?, ?)
        ''', (session_name, history_json, model_name))
        
        self.conn.commit()
        return cursor.lastrowid
    
    def load_session(self, chat_id: int) -> Optional[Dict]:
        """
        Load a chat session by ID.
        
        Args:
            chat_id: The ID of the session to load
            
        Returns:
            Dictionary with session data or None if not found
        """
        cursor = self.conn.cursor()
        cursor.execute('''
            SELECT id, session_name, created_at, updated_at, 
                   model_name, history_json
            FROM chat_sessions 
            WHERE id = ?
        ''', (chat_id,))
        
        row = cursor.fetchone()
        if not row:
            return None
        
        return {
            'id': row['id'],
            'session_name': row['session_name'],
            'created_at': row['created_at'],
            'updated_at': row['updated_at'],
            'model_name': row['model_name'],
            'history': json.loads(row['history_json'])
        }
    
    def update_session(self, chat_id: int, history: List[Dict]):
        """
        Update an existing session with new history.
        
        Args:
            chat_id: ID of the session to update
            history: Updated chat history
        """
        cursor = self.conn.cursor()
        history_json = json.dumps(history)
        
        cursor.execute('''
                UPDATE chat_sessions 
                SET history_json = ?, 
                    updated_at = CURRENT_TIMESTAMP 
                WHERE id = ?
            ''', (history_json, chat_id))
        
        self.conn.commit()
    
    def list_sessions(self, limit: int = 10) -> List[Dict]:
        """
        List recent chat sessions.
        
        Args:
            limit: Maximum number of sessions to return
            
        Returns:
            List of session summaries (without full history)
        """
        cursor = self.conn.cursor()
        cursor.execute('''
            SELECT id, session_name, created_at, updated_at, 
                   model_name
            FROM chat_sessions 
            ORDER BY updated_at DESC 
            LIMIT ?
        ''', (limit,))
        
        sessions = []
        for row in cursor.fetchall():
            sessions.append({
                'id': row['id'],
                'session_name': row['session_name'],
                'created_at': row['created_at'],
                'updated_at': row['updated_at'],
                'model_name': row['model_name']
            })
        
        return sessions
    
    def get_chat_list(self, limit: int = 50) -> List[Dict]:
        """
        Get list of chats for sidebar/history view.
        Returns only chat ID and title, ordered by most recent.
        
        Args:
            limit: Maximum number of chats to return (default 50). Set to None for all chats.
            
        Returns:
            List of dicts with 'chat_id' and 'title' keys
        """
        cursor = self.conn.cursor()
        
        if limit is None:
            # Get all chats
            cursor.execute('''
                SELECT id, session_name 
                FROM chat_sessions 
                ORDER BY updated_at DESC
            ''')
        else:
            # Get limited number
            cursor.execute('''
                SELECT id, session_name 
                FROM chat_sessions 
                ORDER BY updated_at DESC 
                LIMIT ?
            ''', (limit,))
        
        return [
            {'chat_id': row['id'], 'title': row['session_name']}
            for row in cursor.fetchall()
        ]
    
    def delete_session(self, chat_id: int):
        """Delete a session by ID"""
        cursor = self.conn.cursor()
        cursor.execute('DELETE FROM chat_sessions WHERE id = ?', (chat_id,))
        self.conn.commit()
    
    def close(self):
        """Close the database connection"""
        self.conn.close()   
    
    def __enter__(self):
        """Context manager entry"""
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit - closes connection"""
        self.close()

In [ ]:
#|export
db_path="../CHAT.db"
DB = ChatDatabase(db_path)

## Example Usage

In [ ]:
# Save a chat session
history = [
    {'role': 'user', 'content': 'Hello!'},
    {'role': 'assistant', 'content': 'Hi! How can I help you?'},
    {'role': 'user', 'content': 'What is Python?'},
    {'role': 'assistant', 'content': 'Python is a programming language...'}
]

chat_id = DB.save_session(
    session_name="Python discussion",
    history=history,
    model_name="gpt-4"
)

print(f"Saved chat with ID: {chat_id}")

Saved chat with ID: 1


In [ ]:
# Load the session
loaded = DB.load_session(chat_id)
print(f"Session: {loaded['session_name']}")
print(f"Messages: {len(loaded['history'])}")
print(loaded)

Session: Python discussion
Messages: 4
{'id': 1, 'session_name': 'Python discussion', 'created_at': '2025-11-28 18:23:02', 'updated_at': '2025-11-28 18:23:02', 'model_name': 'gpt-4', 'history': [{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hi! How can I help you?'}, {'role': 'user', 'content': 'What is Python?'}, {'role': 'assistant', 'content': 'Python is a programming language...'}]}


In [ ]:
# List all chats
chats = DB.get_chat_list(limit=None)
for chat in chats:
    print(f"{chat['chat_id']}: {chat['title']}")

1: Python discussion


In [ ]:
# Update history
history.append({'role': 'user', 'content': 'Thanks!'})
history.append({'role': 'assistant', 'content': 'You\'re welcome!'})

DB.update_session(chat_id, history)
print("Updated history")

Updated history


In [ ]:
# Delete session (commented out for safety)
# db.delete_session(chat_id)

In [ ]:
DB.get_chat_list()

[{'chat_id': 1, 'title': 'Python discussion'}]

In [ ]:
[i['chat_id']for i in DB.get_chat_list()]

[1]

In [ ]:
# Close connection
DB.close()

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()